In this Jupyter Notebook we want to write a comprehensive script that allows us to tackle inventory management strategy called FEFO, First Expiry First Out.

Let's start with the most basic situation, we have a dictionary of articles and quantities called "Inventory" and we want to iterate through it so to print the data it contains

In [3]:
# Create a variable called `inventory` and store some articles and quantities in it
inventory = {
    "apple": 100,
    "banana": 50,
    "orange": 75
}
# Write a loop to print the article and its quantity.
# We'll use the items() method to access the dictionary..
for k, v in inventory.items():
    print(k, v)


apple 100
banana 50
orange 75


Now we will build upon the `dictionary` object.
Tipically inventory data is stored in relational databases and its format is perfectly represented by that of a `list` of `dictionaries`. Each `dictionary` contains `keys` like 'article' or 'sku_code', 'quantity', 'description' and so on.

In [4]:
# Build a dictionary with three basic data points: 'article', 'quantity' and 'expiration_date'
inventory = [
    {"article": "apple", "quantity": 100, "expiration_date": "2025-01-15"},
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"},
    {"article": "orange", "quantity": 75, "expiration_date": "2025-01-20"}
]

# Learn to iterate through the dictionary accessing each entry or row and thus each `key - value` pair
for entry in inventory:
    print(f"{entry['article']}: {entry['quantity']}, expires on {entry['expiration_date']}")

apple: 100, expires on 2025-01-15
banana: 50, expires on 2025-01-10
orange: 75, expires on 2025-01-20


It is very important at this point that we step out of **Python** and think of the **logic** that we want to implement. Here it goes: 
1. we want to allocate existing inventory to a demand (we haven't yet dealt with demand, but we'll do so very soon) _in order of expiry date_ 
2. we want to dinamically return the remaining inventory after it has been allocated and consumed by production

Let's tackle 1. Ordering the inventory by expiration date. 
We will do so by using a lamda function to sort the inventory. We will also assign the sorted inventory to a new variable which will allow us to keep track of all steps of the way.

In [5]:
# Sort Inventory by Expiration Date
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

print("Sorted Inventory: \n")
for entry in sortinv:
    print(f"{entry['article']}: {entry['quantity']}, expires on {entry['expiration_date']}")

Sorted Inventory: 

banana: 50, expires on 2025-01-10
apple: 100, expires on 2025-01-15
orange: 75, expires on 2025-01-20


We will now build the demand dataset with a very simple list of two items. 
We will keep the same exact structure of the Inventory, i.e. a list of dictionaries, with keys 'article', 'quantity' and a date that tells us when the article is needed by production.

In [6]:
# Build the demand dataset
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},
    {"article": "banana", "quantity": 30, "demand_date": "2025-01-11"}
]

We will also want to order the demand by date, this is because we want to allocate the existing inventory to demand that is closer in time - for later demand lines we will have time to purchase more inventory!

In [7]:
# Sort demand by expiration date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])

print("Sorted Demand: \n")
for requir in sortdem:
    print(f"{requir['article']}: {requir['quantity']}, requirement on {requir['demand_date']}")

Sorted Demand: 

banana: 30, requirement on 2025-01-11
apple: 120, requirement on 2025-01-12


Now that we have both demand and inventory in a good shape, we want to tackle the second problem, i.e. allocating the inventory to the demand and returning the remaining datasets. 

As this is a dinamic and iterative process we will need to use a for loop. And this procedure will essentially do the following: 
- for each line of the demand, take the first line of the inventory with corresponding article (of course we fulfill demand with the correct parts)
- if the demand is larger than the inventory line, then subtract the inventory and reduce it to zero
- - thus search for any other lines in the inventory that can satisfy the remaining demand
- if the demand is less than the inventory, then make the demand zero (fulfill the demand) and store the remaining inventory 

For clarity purposes we will want to delete zero line quantities from the two datasets. 
Also, as before, for clarity purposes, we keep track of all steps of the way by deep-copying our datasets. 

In [8]:
import copy
calc_inventory = copy.deepcopy(sortinv)
calc_demand = copy.deepcopy(sortdem)

# Match demand with existing inventory
for d in calc_demand:
    for i in calc_inventory:
        if i['article'] == d['article']:
            if d['quantity'] >= i['quantity']:
                d['quantity'] -= i['quantity'] # demand - inventory
                i['quantity'] = 0 # thus save the remaining demand
            else:
                i['quantity'] -= d['quantity'] # inventory - demand
                d['quantity'] = 0 # then reduce the demand to zero
calc_demand = [d for d in calc_demand if d['quantity'] > 0]
calc_inventory = [i for i in calc_inventory if i['quantity'] > 0]
# Reproduce the remaining inventory
print("Remaining inventory:")
for i in calc_inventory:
    print(f"{i['article']}: {i['quantity']}")
# Reproduce the remaining demand
print("\nRemaining demand:")
for i in calc_demand:
    print(f"{i['article']}: {i['quantity']}")

Remaining inventory:
banana: 20
orange: 75

Remaining demand:
apple: 20


Once we have made sure that we are able to capture the real life scenario of allocating ordered inventory to ordered demand, we will tackle the next problem, i.e. checking for the additional condition that the inventory is not expired at the time of demand. Essentially, the case where `inventory['expiration_date'] < demand['demand_date']` represents a case where the is no available inventory. Thus we need to exclude the allocation for that case. 

We initialize our sets and want to trigger this behaviour: 
- apples: there is a demand of 120 pieces which cannot be met by the lot expiring on the 10th of January. So that quantity "remains" and is not allocated to production. 
- the 120 demand is satisfied using the second lot, expiring on the 30th of the same month.
- however the subsequent demand of 30 pieces falls short of inventory, (only 130 - 120 = 10 pcs are left)

In [22]:
# initialize sets
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-15"}]
inventory = [
    {"article": "apple", "quantity": 50, "expiration_date": "2025-01-10"}, 
    {"article": "apple", "quantity": 130, "expiration_date": "2025-01-30"}, 
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"}]

# order by date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

# copy for tracking
calc_inv_date = copy.deepcopy(sortinv)
calc_dem_date = copy.deepcopy(sortdem)

We add `and d['demand_date'] < i['expiration_date']` to the conditional. This will ensure proper handling of the expiration and demand dates. 

In [13]:
# Match demand with existing inventory
for d in calc_dem_date:
    for i in calc_inv_date:

        if d['article'] == i['article'] and d['demand_date'] < i['expiration_date']: # if the demand happens prior to the expiration
            if d['quantity'] >= i['quantity']:
                d['quantity'] -= i['quantity'] 
                i['quantity'] = 0 
            else:
                i['quantity'] -= d['quantity'] 
                d['quantity'] = 0 
        # in any other case the demand is unmet (and remains) and the inventory is expired (and remains)
    
# delete zero quanity lines
calc_dem_date = [d for d in calc_dem_date if d['quantity'] > 0]
calc_inv_date = [i for i in calc_inv_date if i['quantity'] > 0]

# Reproduce the remaining inventory
print("Remaining inventory:")
for i in calc_inv_date:
    print(f"{i['article']}: {i['quantity']}")
# Reproduce the remaining demand
print("\nUnmet demand:")
for i in calc_dem_date:
    print(f"{i['article']}: {i['quantity']}")

Remaining inventory:
apple: 50
banana: 50

Unmet demand:
apple: 20


summary tasks
here we are going to display for each line of the inventory:
the initial amount (?), the used up amount and the remaining amount after the allocation
we should also track these for the demand, in case the demand has lines of code that do not appear in the inventory

In [38]:
# initialize sets
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-15"}]
inventory = [
    {"article": "apple", "quantity": 50, "expiration_date": "2025-01-10"}, 
    {"article": "apple", "quantity": 130, "expiration_date": "2025-01-30"}, 
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"}]

# order by date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

# copy for tracking
calc_inv_date = copy.deepcopy(sortinv)
calc_dem_date = copy.deepcopy(sortdem)

# Match demand with existing inventory


for d in calc_dem_date:
    d['consumed_quantity'] = 0 # initialize the consumed quantity to zero for each line
    d['unmet_quantity'] = d['quantity']
    for i in calc_inv_date:
        i['consumed_quantity'] = 0 # initialize the consumed quantity to zero for each line
        i['remaining_quantity'] = i['quantity']

        if d['article'] == i['article'] and d['demand_date'] < i['expiration_date']: # if the demand happens prior to the expiration
            if d['quantity'] >= i['quantity']:
                i['consumed_quantity'] += i['quantity'] # move existing qty to consumed
                i['remaining_quantity'] = 0 # reduce to zero 
                
                d['consumed_quantity'] += i['quantity'] 
                d['unmet_quantity'] -= i['quantity']

            else: # d['quantity'] < i['quantity']
                i['consumed_quantity'] += d['quantity'] # move consumed quantity
                i['remaining_quantity'] -= d['quantity'] 
                
                d['consumed_quantity'] = d['quantity']
                d['unmet_quantity'] = 0 
                
        # in any other case the demand is unmet (and remains) and the inventory is expired (and remains)

inventory_calc = pd.DataFrame(calc_inv_date)
demand_calc = pd.DataFrame(calc_dem_date)
display(inventory_calc)
display(demand_calc)

,article,quantity,expiration_date,consumed_quantity,remaining_quantity
0,apple,50,2025-01-10,0,50
1,banana,50,2025-01-10,0,50
2,apple,130,2025-01-30,30,100


,article,quantity,demand_date,consumed_quantity,unmet_quantity
0,apple,120,2025-01-12,120,0
1,apple,30,2025-01-15,30,0


optimisation of the iterative work: only on the grouped articles

In [70]:
"""
Challenge 1: Group Inventory by Article

Before processing demand, group the inventory items by their article. 
This will let you access relevant inventory directly without iterating through the entire list each time.

Task:
	1.	Create a dictionary where:
	•	Keys are article names.
	•	Values are lists of inventory records for that article.

Example Input:"""
inventory = [
    {"article": "apple", "quantity": 50, "expiration_date": "2025-01-10"},
    {"article": "apple", "quantity": 130, "expiration_date": "2025-01-30"},
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"}
]

"""Expected output:
{
    "apple": [
        {"article": "apple", "quantity": 50, "expiration_date": "2025-01-10"},
        {"article": "apple", "quantity": 130, "expiration_date": "2025-01-30"}
    ],
    "banana": [
        {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"}
    ]
}"""

articles = {}
for i in inventory:
    if i['article'] not in articles: 
        articles[i['article']] = []
        articles[i['article']].append(i)
    else:
        articles[i['article']].append(i) 
articles


{'apple': [{'article': 'apple',
   'quantity': 50,
   'expiration_date': '2025-01-10'},
  {'article': 'apple', 'quantity': 130, 'expiration_date': '2025-01-30'}],
 'banana': [{'article': 'banana',
   'quantity': 50,
   'expiration_date': '2025-01-10'}]}

now we use the grouped inventory `article` to process the allocation

In [74]:
# initialize sets
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-15"}]
inventory = [
    {"article": "apple", "quantity": 50, "expiration_date": "2025-01-10"}, 
    {"article": "apple", "quantity": 130, "expiration_date": "2025-01-30"}, 
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"}]

# order by date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

# copy for tracking
import copy
calc_inv_date = copy.deepcopy(sortinv)
calc_dem_date = copy.deepcopy(sortdem)

# process a grouped inventory to run the calculation faster
articles = {}
for i in calc_inv_date:
    if i['article'] not in articles: 
        articles[i['article']] = []
        articles[i['article']].append(i)
    else:
        articles[i['article']].append(i) 


# Match demand with existing inventory
for d in calc_dem_date:
    d['consumed_quantity'] = 0 # initialize the consumed quantity to zero for each line
    d['unmet_quantity'] = d['quantity']
    for i in calc_inv_date:
        i['consumed_quantity'] = 0 # initialize the consumed quantity to zero for each line
        i['remaining_quantity'] = i['quantity']

        if d['article'] == i['article'] and d['demand_date'] < i['expiration_date']: # if the demand happens prior to the expiration
            if d['quantity'] >= i['quantity']:
                i['consumed_quantity'] += i['quantity'] # move existing qty to consumed
                i['remaining_quantity'] = 0 # reduce to zero 
                
                d['consumed_quantity'] += i['quantity'] 
                d['unmet_quantity'] -= i['quantity']

            else: # d['quantity'] < i['quantity']
                i['consumed_quantity'] += d['quantity'] # move consumed quantity
                i['remaining_quantity'] -= d['quantity'] 
                
                d['consumed_quantity'] = d['quantity']
                d['unmet_quantity'] = 0 
                
        # in any other case the demand is unmet (and remains) and the inventory is expired (and remains)

import pandas as pd
inventory_calc = pd.DataFrame(calc_inv_date)
demand_calc = pd.DataFrame(calc_dem_date)
display(inventory_calc)
display(demand_calc)

,article,quantity,expiration_date,consumed_quantity,remaining_quantity
0,apple,50,2025-01-10,0,50
1,banana,50,2025-01-10,0,50
2,apple,130,2025-01-30,30,100


,article,quantity,demand_date,consumed_quantity,unmet_quantity
0,apple,120,2025-01-12,120,0
1,apple,30,2025-01-15,30,0


In [81]:
for i in calc_inv_date:
    if i['article'] == 'apple':
        print(i)

{'article': 'apple', 'quantity': 50, 'expiration_date': '2025-01-10', 'consumed_quantity': 0, 'remaining_quantity': 50}
{'article': 'apple', 'quantity': 130, 'expiration_date': '2025-01-30', 'consumed_quantity': 30, 'remaining_quantity': 100}


In [91]:
for k, v in articles.items():
    if k == 'apple':
        for i in v:
            print(i)

{'article': 'apple', 'quantity': 50, 'expiration_date': '2025-01-10', 'consumed_quantity': 0, 'remaining_quantity': 50}
{'article': 'apple', 'quantity': 130, 'expiration_date': '2025-01-30', 'consumed_quantity': 30, 'remaining_quantity': 100}


In [101]:
# initialize sets
demand = [
    {"article": "apple", "quantity": 120, "demand_date": "2025-01-12"},
    {"article": "apple", "quantity": 30, "demand_date": "2025-01-15"}]
inventory = [
    {"article": "apple", "quantity": 50, "expiration_date": "2025-01-10"}, 
    {"article": "apple", "quantity": 130, "expiration_date": "2025-01-30"}, 
    {"article": "banana", "quantity": 50, "expiration_date": "2025-01-10"}]

# order by date
sortdem = sorted(demand, key=lambda demd: demd['demand_date'])
sortinv = sorted(inventory, key=lambda expi: expi['expiration_date'])

# copy for tracking
import copy
calc_inv_date = copy.deepcopy(sortinv)
calc_dem_date = copy.deepcopy(sortdem)

# process a grouped inventory to run the calculation faster
articles = {}
for i in calc_inv_date:
    if i['article'] not in articles: 
        articles[i['article']] = []
        articles[i['article']].append(i)
    else:
        articles[i['article']].append(i) 

"""here we have to place the search by article in the grouped list, instead of the entire list"""

# Match demand with existing inventory
for d in calc_dem_date:
    d['consumed_quantity'] = 0 # initialize the consumed quantity to zero for each line
    d['unmet_quantity'] = d['quantity']
    if d['article'] in articles:
        
        for k, v in articles.items():
            v['consumed_quantity'] = 0 # initialize the consumed quantity to zero for each line
            v['remaining_quantity'] = v['quantity']
            
            for i in v:
                

                if d['article'] == i['article'] and d['demand_date'] < i['expiration_date']: # if the demand happens prior to the expiration
                    if d['quantity'] >= i['quantity']:
                        i['consumed_quantity'] += i['quantity'] # move existing qty to consumed
                        i['remaining_quantity'] = 0 # reduce to zero 
                        
                        d['consumed_quantity'] += i['quantity'] 
                        d['unmet_quantity'] -= i['quantity']

                    else: # d['quantity'] < i['quantity']
                        i['consumed_quantity'] += d['quantity'] # move consumed quantity
                        i['remaining_quantity'] -= d['quantity'] 
                        
                        d['consumed_quantity'] = d['quantity']
                        d['unmet_quantity'] = 0 







import pandas as pd
inventory_calc = pd.DataFrame(calc_inv_date)
demand_calc = pd.DataFrame(calc_dem_date)
display(inventory_calc)
display(demand_calc)

TypeError: list indices must be integers or slices, not str